In [ ]:
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet50, inception_v3, googlenet, vgg16, mobilenet_v2
from saliency.saliency_zoo import fast_ig, guided_ig, big, agi, ig, ISA, saliencymap, sm, sg, saliencymap, deeplift
from tqdm import tqdm
import torch
import numpy as np
import argparse
import torch
import random

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(3407)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = resnet50(pretrained=True).eval().to(device)
sm = nn.Softmax(dim=-1)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
norm_layer = transforms.Normalize(mean, std)
sm = nn.Softmax(dim=-1)
model = nn.Sequential(norm_layer, model, sm).to(device)


In [ ]:
img_batch = torch.load("data/img_batch.pt") # img_batch.shape = (1000,3,224,224)
target_batch = torch.load("data/label_batch.pt") # target_batch.shape = (1000,)
batch_size = 128
attributions = []
for i in tqdm(range(0, len(img_batch), batch_size)):
    img = img_batch[i:i+batch_size].to(device)
    target = target_batch[i:i+batch_size].to(device)
    attributions.append(mfaba_sharp(model, img, target))
if attributions[0].shape.__len__() == 3:
    attributions = [np.expand_dims(attribution, axis=0) for attribution in attributions]
attributions = np.concatenate(attributions, axis=0)